# Dive to Normalizing Flows
---

## Вывод нормализующих потоков

Давайте посмотрим еще раз на этот рисунок и попробуем расписать итерационный процесс

<figure>
<img src="https://lilianweng.github.io/posts/2018-10-13-flow-models/normalizing-flow.png" alt="Примеры схем генеративных моделй" style="width:100%">
<figcaption align = "center">Иллюстрация преобразования нормально распределенного z0 в zK из реального  распределения.</figcaption>
</figure>

Положим, существуют
$$
z_{i-1}\sim p_{i-1}(z_{i-1}), ~ z_i = f_i(z_{i-1})\Leftrightarrow z_{i-1} = f^{-1}_i(z_i)
$$

где $z_{i-1}$ — вектор, распределенный как $p_{i-1}$, а $f_i$ — функция, отображающая $z_{i-1}$ в $z_i$. Также существует обратная к $f$ функция $f^{-1}$, которая отображает в обратную сторону.

Переход потграфу на рисунке можно расписать

$$
\begin{split}
p_i(z_i) & \stackrel{(1)}{=} p_{i-1}(f_i^{-1}(z_i))\left|\det\left(\dfrac{df^{-1}_i(z_i)}{dz_i}\right)\right| \\
& \stackrel{(2)}{=} p_{i-1}(z_{i-1})\left|\det\left(\dfrac{df_i(z_{i-1})}{dz_{i-1}}\right)^{-1}\right| \\
& \stackrel{(3)}{=} p_{i-1}(z_{i-1})\left|\det\left(\dfrac{df_i(z_{i-1})}{dz_{i-1}}\right)\right|^{-1} \\
\end{split}
$$

Разберем подробнее каждый переход

1. Получено из теоремы о замене переменной для многомерных случайных величин.
2. Следует из теоремы об обратимых функциях.
3. Для обратимых матриц существует свойство определителя $\det(A^{-1}) = \det(A)^{-1}$.

### Теорема о замене переменной

Положим существует переменная $z$ распределенная как $z\sim\pi(z)$, при помощи которой мы хотим получить новую независимую переменную, используя преобразование $x=f(z)$. Функцию $f$ выберем так, что $\exists f^{-1}: z=f^{-1}(x)$. Вопрос, каким будет распределение $x\sim p(x)$?
$$
\int p(x)dx=\int\pi(z)dz = 1
$$

$$
p(x) = \pi(z)\left|\dfrac{dz}{dx}\right| = \pi(f^{-1}(x))\left|\dfrac{df^{-1}}{dx}\right| = \pi(f^{-1}(x))|(f^{-1})^{'}(x)|
$$

В случае многомерной случайной величины выражение схоже

$$
\vec{z}\sim\pi(\vec{z}), ~\vec{x}=f(\vec{z}),~\vec{z}=f^{-1}(\vec{x})
$$

$$
p(\vec{x}) = \pi(\vec{z})\left|det\dfrac{d\vec{z}}{\vec{x}}\right| = \pi(f^{-1}(\vec{x}))\left|det\dfrac{df^{-1}(\vec{x})}{\vec{x}}\right|
$$

где $det\frac{df^{-1}(\vec{x})}{d\vec{x}}$ — определитель Якобиана функции $f^{-1}$.

### Теорема об обратных функциях
Теорема об обратных функциях говорит о том, что если $y = f(x),~x=f^{-1}(y)$, то
$$
\dfrac{df^{-1}(y)}{dy} = \dfrac{dx}{dy}=\left(\dfrac{dy}{dx}\right)^{-1} = \left(\dfrac{df(x)}{dx}\right)^{-1}
$$

### Свойство определителя обратной матрицы
$$
A\cdot A^{-1} = I \Rightarrow \det(A\cdot A^{-1}) = \det(I)  \Rightarrow \det(A\cdot A^{-1}) = 1 \\
\det(A)\cdot \det(A^{-1}) = 1 \Rightarrow \det(A^{-1}) = \dfrac{1}{\det(A)} = \det(A)^{-1}
$$

Таким образом получить из нормально распределенных $z_0$ объекты $x=z_K$ можно через суперпозицию всех функций $f_i$ как
$$
x=z_K=f_K\circ f_{K-1}\circ\cdots\circ f_1(z_0)
$$

Тогда трансформация логарифма плотности распределения
$$
\begin{split}
\log p(x) = \log\pi_K(z_K) & = \log\pi_{K-1}(z_{K-1})-\log\left|\det\dfrac{df_K}{dz_{K-1}}\right| \\
& = \log\pi_{K-2}(z_{K-2}) -\log\left|\det\dfrac{df_{K-1}}{dz_{K-2}}\right| -\log\left|\det\dfrac{df_K}{dz_{K-1}}\right| \\
& = \log\pi_0(z_0) - \sum\limits_{i=1}^K\log\left|\det\dfrac{df_i}{dz_{i-1}}\right|
\end{split}
$$

Здесь преобразование $z_i=f_i(z_{i-1})$ называется потоком (flow), а полная последовательность из преобразований $\pi_i$ называется нормализующим потоком (normalizing flow).

Стоит уточнить, что на функции $f_i$ накладывается ряд требований:

1. $f_i$ должна быть легко обратимой функцией
2. Якобиан $\frac{df_i}{dz_{i-1}}$ должен быть легко вычислимым


## Affine Coupling Layer
$$
\begin{cases}
y_i = x_i,~\forall i=1,2,\dots,d \\
y_i = x_i \odot \exp(s(x_{i-d})) + t(x_{i-d}),~\forall i=d+1,\dots,D
\end{cases}
$$

где $s(\cdot)$ — масштабирование, $t(\cdot)$ — смещение, а $\odot$ — поэлементное умножение.

Теперь убедимся, что это преобразование подходит нам по критериям функции $f$.

### Обратимость
$$
\begin{cases}
y_i = x_i~,\\
y_i = x_i \odot \exp(s(x_{i-d})) + t(x_{i-d})~;
\end{cases} \Leftrightarrow
\begin{cases}
x_i = y_i~,\\
x_i = (y_i - t(x_{i-d})) \odot \exp(-s(x_{i-d}))~.
\end{cases}
$$

Так как преобразование линейно, оно не требует поиска обратных функций к $s(\cdot)$ и $t(\cdot)$, а значит, что вычисляется быстро.

### Определитель Якобиана

Легко заметить, что Якобиан функции имеет вид нижней треугольной матрицы

$$
J =
\begin{bmatrix}
\mathbf{I} & \mathbf{0} \\
\frac{\partial y_{d+1:D}}{\partial x^T_{1:d}} & \text{diag}(\exp[s(x_{1:d})])
\end{bmatrix}
$$

где
- $\mathbf{I}_d$ — единичная матрица, полученная из $\frac{\partial y_{1:d}}{\partial x_{1:d}} = \frac{\partial x_{1:d}}{\partial x_{1:d}}$ размерности $d\times d$
- $\mathbf{0}$ — нулевая матрица, полученная из $\frac{\partial y_{1:d}}{\partial x_{d+1:D}} = \frac{\partial x_{1:d}}{\partial x_{d+1:D}}$ размерности $d\times (D-d)$
- $\frac{\partial y_{d+1:D}}{\partial x^T_{1:d}}$ — минор Якобиана размерности $(D-d)\times d$
- $\text{diag}(\exp[s(x_{1:d})])$ — диагональная матрица, где на диагонали значения $\exp[s(x_{1:d})]$, а вне — нули. Размерность это матрицы $(D-d)\times(D-d)$

Можно заметить, что такая матрица может существовать только при $d = \frac{D}{2}$.

Так как матрица имеет вид нижней треугольно, ее определитель считается по диагонали, а значит, что левую нижнюю часть матрицы знать не обязательно. Тогда определитель Якобиана вычисляется как

$$
\det(J) = \prod\limits_{j=1}^{D-d}\exp(s(x_{1:d}))_j = \exp\left(\sum\limits_{j=1}^{D-d}s(x_{1:d})_j\right)
$$

Так как вычисление определителя сводится к вычислению экспоненты суммы выходов $s(\cdot)$, считаться определитель будет быстро.

Так как к функциям $s(\cdot)$ и $t(\cdot)$ не выставлено никаких сложных требований, в их качестве можно использовать функции класса нейронных сетей.

### Использование маски

Так как маска влияет на порядок строк и столбцов в матрице Якобиана нужно сказать, что перестановка строк или столбцов в определители влияет только на знак определителя матрицы, а так как для вычисления плотности распределения мы используем модуль определителя, перестановка строк и столбцов на вычисление плотности не влияет.

Таким образом мы можем свести вычисление оперделителя при использовании маски к базовому определению афинного потока выше, переставляя строки матрицы.

## Деквантизация

Нормализующие потоки полагаются на правило изменения переменных, которое естественным образом определено в непрерывном пространстве. Применение потоков непосредственно к дискретным данным приводит к нежелательным моделям плотности, в которых произвольно высокая вероятность присваивается нескольким конкретным значениям.

Из-за разницы в свойствах распределений дискретного и непрерывного типов плотность получается неправильной (интеграл не равен единице). С целью исправить это, существует операция деквантизации.

<figure>
<img src="https://github.com/sswt/dive2gai/blob/unit3/unit3/imgs/simple_flow.png" alt="Примеры схем генеративных моделй" style="width:100%">
<figcaption align = "center">Пример работы нормализующего потока без квантизации с дискретным распределением.</figcaption>
</figure>

### Обычная деквантизация

Деквантизированную величину обозначим как $v$. тогда операция деквантизации модет быть сформулирована как $v = x + u,~u\in[0, 1)^D$. Тогда наша плотность описывается как

$$
p(v) = \int p(x+u)du = \int \dfrac{q(u|x)}{q(x|x)}p(x+u)du = \mathbb{E}_{u\sim q(u|x)}\left[\dfrac{p(x+u)}{q(u|x)}\right] = \mathbb{E}_{u\sim U(0, 1)^D}[p(x+u)]
$$

Далее нам нужно применить сигмоидальное преобразование $\sigma(v)$. Обратное преобразование $\sigma(v)^{-1}$ имеет вид

$$
\sigma(v)^{-1} = \log z - \log 1-z
$$


<figure>
<img src="https://github.com/sswt/dive2gai/blob/unit3/unit3/imgs/dequant_flow.png" alt="Примеры схем генеративных моделй" style="width:100%">
<figcaption align = "center">Пример работы нормализующего потока с простой квантизацией.</figcaption>
</figure>

### Вариационная деквантизация

Для деквантизации мы используем нормальное распределение, что может быть некорректно, когда дискретные величины изначально не равновзвешаны. С целью исправить это вводится вариационная деквантизация

<figure>
<img src="https://github.com/sswt/dive2gai/blob/unit3/unit3/imgs/var_deq_flow.png" alt="Примеры схем генеративных моделй" style="width:100%">
<figcaption align = "center">Пример работы нормализующего потока с вариационной деквантизацией.</figcaption>
</figure>